In [1]:
import torch

In [2]:
torch.backends.mps.is_available()

True

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [62]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        left = 0
        right = 1
        char_index = {s[left]}
        max_length=1
        while right<len(s):
            if s[right] not in char_index:
                if right-left+1>max_length:
                    max_length = right-left+1
                char_index.add(s[right])
                right+=1   
            else:
                char_index.remove(s[left])
                left+= 1
        return max_length
            
            

In [65]:
Solution().lengthOfLongestSubstring('abc')

3

In [39]:
test = set()
test.add(43)
test.add('d')
test.remove(43)
test
test.remove('sdfsdff')


KeyError: 'sdfsdff'

In [7]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=1,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/Users/htong/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 1, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?
 nobody


In [3]:
len(tokenizer.encode('I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n').split(' '))

NameError: name 'tokenizer' is not defined

In [7]:
from transformers import AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 512)#training_config["model"]["max_length"])
      )
    }
  )
  # * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):